In [1]:
import os
import time
import math
import json
import redis
import random
import asyncio
import traceback
from datetime import datetime
from aredis import StrictRedis

In [2]:
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [3]:
from libs.utils import Utils
from libs.dailydata import DailyData
from libs.cython.compute import compute_stats

In [4]:
rd = redis.Redis(host='127.0.0.1', port=6379, db=8)
ar = StrictRedis(host='127.0.0.1', port=6379, db=8)

In [5]:
Utils.update_symbols()
symbols = Utils.get_running_symbols()
assist_count = math.ceil(len(symbols)/800)+1
rd.set(f'hq_assist_count', assist_count)

True

In [6]:
date = time.strftime('%Y%m%d')
date = '20210812'
date

'20210812'

In [7]:
dd = DailyData.load(dt=date)

In [11]:
dd.close_sharedmemory()
dd.unlink_sharedmemory()

In [8]:
async def start_fake_data_generator(assist_count=7, snapshot_interval=5):
    assist_indices = list([ _ for _ in range(assist_count)])
    
    for _, check_point in enumerate(dd.check_points):
        random.shuffle(assist_indices)
        try:
            time_lapse = dd.get_time_lapse(_)
            ma5pm_anchor_idx = dd.get_ma5pm_anchor_idx(_)
            fs5p = dd.snapshots[ma5pm_anchor_idx]
            compute_stats(dd.snapshots[_], dd.basics, dd.statistic[_], fs5p, time_lapse)

            for assist_idx in assist_indices:
                await ar.publish(
                    f'hq_assist_{assist_idx}_snapshotting', 
                    json.dumps({"status":'successful',"date":date,"idx":_,'check_point':int(check_point)})
                )
                
            print(str(_).ljust(5, ' '), datetime.fromtimestamp(dd.check_points[_]).strftime('%Y-%m-%d %H:%M:%S'), 'sussess', end='\r')

        except Exception as e:
            error = {
                "status": 'failed',
                "idx": _,
                'check_point':int(check_point),
                "exception": str(e),
                'traceback': traceback.format_exc()
            }

            for assist_idx in assist_indices:
                await ar.publish(
                    f'hq_assist_{assist_idx}_snapshotting', 
                    json.dumps(error)
                )
            print(str(_).ljust(5, ' '), datetime.fromtimestamp(dd.check_points[_]).strftime('%Y-%m-%d %H:%M:%S'), 'failed', str(e), end='\r')
        finally:
            if _ > 190:
                await asyncio.sleep(snapshot_interval)


In [11]:
task = asyncio.create_task(start_fake_data_generator())

In [10]:
task.cancel()

True

In [ ]:
import numpy as np

In [ ]:
_ = 3006

In [ ]:
dd.get_snapshot(_)

In [ ]:
ma5pm_anchor_idx = dd.get_ma5pm_anchor_idx(_)
ma5pm_anchor_idx

In [ ]:
dd.get_snapshot(2945)

In [ ]:
time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(1628492100))

In [ ]:
with open(os.path.join(os.getcwd(), 'config.json')) as file:
    config = json.load(file)
config

In [ ]:
aredis = StrictRedis(
    host=config['redis']['host'],
    port=config['redis']['port'],
    db=config['redis']['db'],
    password=config['redis']['password'],
    decode_responses=True
)

In [ ]:
await aredis.get('temp')

In [ ]:
await aredis.delete('temp')

In [ ]:
idx = 2661

In [ ]:
result = dd.statistic[idx, :, 4] > 0
result

In [ ]:
len(np.argwhere(result))

In [ ]:
result2 = dd.statistic[idx-1, :, 4] <= 0
result2

In [ ]:
len(np.argwhere(result2))

In [ ]:
np.argwhere(result == result2)

In [ ]:
result2[3448]

In [ ]:
dd.check_points[np.argmax(dd.statistic[:2703, 3348, 4] > 0)]

In [ ]:
time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(float(ts)))

In [ ]:
dd.statistic[1000,:,4]

In [ ]:
import numpy as np

In [ ]:
[
    [
        idx[0], 
        dd.check_points[np.argmax(dd.statistic[:,idx[0],4]>0)]
    ] for idx in np.argwhere(dd.statistic[2000,:,4]>0)
]


In [ ]:
start = time.time()
json.dumps(list([
    (
        int(idx[0]), 
        int(dd.check_points[np.argmax(dd.statistic[:,idx[0],4]>0)])
    ) for idx in np.argwhere(dd.statistic[2103,:,4]>0)
]))
time.time()-start

In [ ]:
delta = 1926
time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(1626073210+delta))

In [ ]:
dd.check_points[2103]

In [ ]:
1626052500+delta

In [ ]:
dd.check_points[-1]

In [ ]:
import numpy as np

In [ ]:
np.argwhere(dd.statistic[1358, :, 4] > 0).tolist()

In [ ]:
dd.check_points[1358]